# ETL

Se nos han dado tres archivos con los cuales debemos trabajar. Estos archivos contienen información sobre los juegos de Steam, usuarios, items y reviews. Debemos extraer esta información de los archivos recibidos y trabajarla para que esté disponible en DataFrames de pandas.

Primero debo importar las librerías necesarias para el proceso de ETL

In [1]:
import pandas as pd
import ast
import json
import warnings
warnings.filterwarnings('ignore')

### 1. Archivo steam_games.json
El trabajo de ETL con este archivo consiste en abrirlo, almacenar la info en un DataFrame de pandas, inspeccionar los datos, eliminar registros con datos nulos y repetidos. Analizar otros cambios o transformaciones que sea necesario realizar, definir tipo de datos de las variables, eliminar columnas que no necesite.

In [2]:
# Leo el archivo steam_games.json y lo guardo en un dataframe df_games
df_games=pd.read_json('Dataset/steam_games.json', lines=True)

#### Observación del contenido extraído del archivo

In [3]:
# Ejecuto info para tener más información sobre las columnas extraídas del archivo y los tipos de 
#datos interpretados
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [4]:
df_games.shape

(120445, 13)

In [5]:
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [6]:
df_games.tail()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None


#### Inspección de valores nulos

In [7]:
# Veo que hay muchas filas sin datos, entonces hago un conteo de valores nulos por columna
df_games.isnull().sum()

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [8]:
df_games2=df_games.copy()

In [9]:
# Elimino las filas que tienen todos sus datos nulos
df_games2.dropna(how='all', inplace=True)
df_games2.reset_index(inplace=True, drop=True)
df_games2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.2+ MB


#### Revisión por ID

In [10]:
# Verifico si existen registros sin id
rows_with_nan_id = df_games2[df_games2['id'].isna()]
rows_with_nan_id

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
74,None,None,None,None,http://store.steampowered.com/,None,None,None,None,19.99,0.0,NaN,None
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [11]:
""" Eliimino los registros sin id, ya que es un dato fundamental para resolver las consultas posteriores.
Además son sólo dos registros, así que eliminarlos no afectará de forma notoria los resultados """
df_games2 = df_games2.drop(df_games2.index[74])
df_games2 = df_games2.drop(df_games2.index[30961])

In [12]:
# Reviso si existen registros con id duplicado
print("Valores duplicados en Id:", df_games2['id'].duplicated().sum())

Valores duplicados en Id: 1


In [13]:
# Se eliminan los valores duplicados
df_games2 = df_games2.drop_duplicates(subset="id")

#### Revisión por columnas Publisher y Developer

Las columnas Publisher y Developer tienen información similar. Developer tiene más datos notnull que Publisher, por eso decido quedarme con esa columna. Antes de eliminar Publisher voy a buscar si hay informacion allí que me sirva para completar faltantes en Developer

In [14]:
# Identifico las filas que no tienen datos en developer pero sí en publisher, hago una lista con los índices
nuevas_filas = df_games2[df_games2['developer'].isnull() & ~df_games2['publisher'].isnull()]
nuevas_filas
indices=nuevas_filas.index.tolist()


# Siguiendo la lista de índices, a esos registros les asigno el valor de publisher en la columna developer
for i in indices:
    df_games2['developer'][i]=df_games2['publisher'][i]

In [15]:
# Reviso el dataframe después de los cambios realizados
df_games2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 0 to 32134
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24081 non-null  object 
 1   genres        28850 non-null  object 
 2   app_name      32131 non-null  object 
 3   title         30083 non-null  object 
 4   url           32132 non-null  object 
 5   release_date  30066 non-null  object 
 6   tags          31970 non-null  object 
 7   reviews_url   32131 non-null  object 
 8   specs         31463 non-null  object 
 9   price         30755 non-null  object 
 10  early_access  32132 non-null  float64
 11  id            32131 non-null  float64
 12  developer     28899 non-null  object 
dtypes: float64(2), object(11)
memory usage: 4.4+ MB


Puedo ver que los valores notnull en la columna publisher pasaron de ser 28836 a ser 28899

In [16]:
# Elimino la columna publisher
df_games2.drop(['publisher'], axis=1, inplace=True)
df_games2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 0 to 32134
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28850 non-null  object 
 1   app_name      32131 non-null  object 
 2   title         30083 non-null  object 
 3   url           32132 non-null  object 
 4   release_date  30066 non-null  object 
 5   tags          31970 non-null  object 
 6   reviews_url   32131 non-null  object 
 7   specs         31463 non-null  object 
 8   price         30755 non-null  object 
 9   early_access  32132 non-null  float64
 10  id            32131 non-null  float64
 11  developer     28899 non-null  object 
dtypes: float64(2), object(10)
memory usage: 4.2+ MB


#### Revisión por columnas App name y Title

Estas dos columnas tienen información similar, puedo revisarlas para al final quedarme con una sola de las dos

In [17]:
# Verifico si existen registros sin nombre
rows_with_nan_app_name = df_games2[df_games2['app_name'].isna()]
rows_with_nan_app_name

,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
2580,"[Action, Indie]",None,None,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",None,0.0,317160.0,None


In [18]:
# Puedo conseguir el nombre del juego con la url y llenar ese campo vacío
print(df_games2['app_name'][2580])
print(df_games2['url'][2580])
print(df_games2['title'][2580])
df_games2['app_name'][2580]= 'Duet'

None
http://store.steampowered.com/app/317160/_/
None


In [19]:
# Verifico si existen registros sin título
rows_with_nan_title = df_games2[df_games2['title'].isna()]
rows_with_nan_title
print(len(rows_with_nan_title))

2049


Como todos los registros tienen app_name decido eliminar la columna title que tiene muchos valores nulos


In [20]:
# Elimino la columna title
df_games2.drop(['title'], axis=1, inplace=True)
df_games2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 0 to 32134
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28850 non-null  object 
 1   app_name      32132 non-null  object 
 2   url           32132 non-null  object 
 3   release_date  30066 non-null  object 
 4   tags          31970 non-null  object 
 5   reviews_url   32131 non-null  object 
 6   specs         31463 non-null  object 
 7   price         30755 non-null  object 
 8   early_access  32132 non-null  float64
 9   id            32131 non-null  float64
 10  developer     28899 non-null  object 
dtypes: float64(2), object(9)
memory usage: 3.9+ MB


#### Revisión columna url

In [21]:
# Verifico que no haya urls duplicadas, porque cada juego tiene su propia url, entonces url duplicados 
#significa juegos duplicados
has_duplicates = df_games2['url'].duplicated()
total_duplicates = has_duplicates.sum()
print(total_duplicates)

0


#### Revisión columna release date

De la fecha completa solo voy a necesitar el año de lanzamiento del juego, entonces extraigo ese dato en una nueva columna y elimino la columna con las fechas completas

In [22]:
# Se extrae el año en una nueva columna llamada "release_year" a través del uso de expresiones regulares
df_games2['release_year'] = df_games2['release_date'].str.extract(r'(\d{4})')

# Se elimina la columna original
df_games2 = df_games2.drop(columns="release_date")
df_games2.head()

,genres,app_name,url,tags,reviews_url,specs,price,early_access,id,developer,release_year
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro,2018
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL,2018
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com,2017
3,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域,2017
4,None,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None,NaN


#### Elimino columnas que no voy a necesitar: url, tags, specs, reviews_url, early_access. Esta decisión la tomo teniendo en cuenta los endpoints y las consultas que debo realizar.

In [23]:
df_games2=df_games2.drop(columns=["url", "reviews_url","specs", "tags", "early_access"])

Observo las características del dataset que tengo en este momento

In [24]:
df_games2.columns

Index(['genres', 'app_name', 'price', 'id', 'developer', 'release_year'], dtype='object')

In [25]:
df_games2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 0 to 32134
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28850 non-null  object 
 1   app_name      32132 non-null  object 
 2   price         30755 non-null  object 
 3   id            32131 non-null  float64
 4   developer     28899 non-null  object 
 5   release_year  29965 non-null  object 
dtypes: float64(1), object(5)
memory usage: 2.7+ MB


In [26]:
# Hago un nuevo filtrado de valores nulos y  reconstruyo el indice
df_games2 = df_games2.dropna().reset_index(drop=True)
df_games2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27508 entries, 0 to 27507
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        27508 non-null  object 
 1   app_name      27508 non-null  object 
 2   price         27508 non-null  object 
 3   id            27508 non-null  float64
 4   developer     27508 non-null  object 
 5   release_year  27508 non-null  object 
dtypes: float64(1), object(5)
memory usage: 1.3+ MB


#### Revisión columna price
Esta columna tiene valores numéricos y de texto. Necesito unificar el tipo de datos.

In [27]:
# Busco los valores únicos
precios = df_games2['price'].unique()
precios

array([4.99, 'Free To Play', 'Free to Play', 0.99, 3.99, 9.99, 18.99,
       29.99, 10.99, 2.99, 1.5899999999999999, 14.99, 1.99, 59.99, 8.99,
       6.99, 7.99, 39.99, 'Free', 19.99, 7.49, 12.99, 5.99, 2.49, 15.99,
       1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99,
       79.99, 49.99, 5.0, 13.98, 29.96, 109.99, 149.99, 771.71,
       'Install Now', 21.99, 89.99, 'Play WARMACHINE: Tactics Demo', 0.98,
       139.92, 4.29, 'Free Mod', 54.99, 64.99, 74.99, 'Install Theme',
       0.89, 'Third-party', 0.5, 'Play Now', 299.99, 1.29, 119.99, 44.99,
       3.0, 15.0, 5.49, 23.99, 49.0, 10.93, 1.3900000000000001,
       'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0, 4.0,
       1.9500000000000002, 1.5, 199.0, 189.0, 6.66, 27.99, 129.99, 179.0,
       26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 22.99, 320.0, 38.85,
       71.7, 995.0, 27.49, 3.39, 6.0, 19.95, 20.99, 499.99, 199.99, 16.06,
       4.68, 

In [28]:
""" Defino una nueva lista para almacenar, de los valores encontrados en el paso anterior, 
los que no son valores numéricos """
no_numeros = []
for valor in precios:
    try:
        float(valor)
    except ValueError:
        no_numeros.append(valor)

no_numeros

['Free To Play',
 'Free to Play',
 'Free',
 'Free Demo',
 'Play for Free!',
 'Install Now',
 'Play WARMACHINE: Tactics Demo',
 'Free Mod',
 'Install Theme',
 'Third-party',
 'Play Now',
 'Free HITMAN™ Holiday Pack',
 'Play the Demo',
 'Free to Try',
 'Free to Use']

In [29]:
""" Observando la anterior lista, la mayoría parecen sugerir que son pruebas grtuitas o demos, 
decido reemplazan esos valores por cero """
df_games2['price'] = df_games2['price'].replace(no_numeros, '0.0')

# Se convierte la columna a tipo "float"
df_games2['price'] = df_games2['price'].astype(float)
df_games2.head()

,genres,app_name,price,id,developer,release_year
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,4.99,761140.0,Kotoshiro,2018
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,0.00,643980.0,Secret Level SRL,2018
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,0.00,670290.0,Poolians.com,2017
3,"[Action, Adventure, Casual]",弹炸人2222,0.99,767400.0,彼岸领域,2017
4,"[Action, Adventure, Simulation]",Battle Royale Trainer,3.99,772540.0,Trickjump Games Ltd,2018


#### Revisión columna genres
Para cumplir con algunos de los requerimientos del proyecto, necesito expandir la columna genres, con los procedimientos que desarrollaré a partir de este punto

In [30]:
""" Voy a expandir la columna genres utilizando el metodo explode, y a generar un dataframe individual con 
los datos de esa nueva columna expandida y la columna id, que me servirá para unir de nuevo esta información
al dataframe original """
df_games_genres=df_games2[['genres', 'id']]
df_games_genres= df_games_genres.explode('genres')

# Se consultan los valores únicos
unique_genres = df_games_genres['genres'].unique()
unique_genres

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing',
       'Early Access', 'Massively Multiplayer',
       'Animation &amp; Modeling', 'Web Publishing', 'Education',
       'Software Training', 'Utilities', 'Design &amp; Illustration',
       'Audio Production', 'Video Production', 'Photo Editing',
       'Accounting'], dtype=object)

In [31]:
len(unique_genres)

22

In [32]:
# Creo variables dummy para genres (col)
df_games_genres = pd.get_dummies(data = df_games_genres, columns=["genres"], dtype = int, prefix="", prefix_sep = "")

print(df_games_genres.shape)
df_games_genres.head(2)

(68127, 23)


,id,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,761140.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,761140.0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:

# Se observa la cantidad de filas en el DF original para comparar
df_games2.shape

(27508, 6)

Lo que interpreto es que hay 27508 juegos, así que debo agrupar los registros que obtuve al generar los dummies (68127), para que, en cada registro, quede almacenada la información de todos los generos que le corresponden a cada juego según su Id

In [34]:
# Se agrupan los generos por Id
df_games_genres = df_games_genres.groupby("id").sum().reset_index()

df_games_genres.shape

(27508, 23)

In [35]:
# Se unen el DataFrame original y este nuevo, según el Id
df_steam_games = pd.merge(df_games2, df_games_genres, on = "id", how = "inner")
df_steam_games.head()

,genres,app_name,price,id,developer,release_year,Accounting,Action,Adventure,Animation &amp; Modeling,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,4.99,761140.0,Kotoshiro,2018,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,0.00,643980.0,Secret Level SRL,2018,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,0.00,670290.0,Poolians.com,2017,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,"[Action, Adventure, Casual]",弹炸人2222,0.99,767400.0,彼岸领域,2017,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"[Action, Adventure, Simulation]",Battle Royale Trainer,3.99,772540.0,Trickjump Games Ltd,2018,0,1,1,0,...,0,0,0,1,0,0,0,0,0,0


In [36]:
# Elimino la columna 'genres'
df_steam_games=df_steam_games.drop(columns='genres')

In [37]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27508 entries, 0 to 27507
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   app_name                   27508 non-null  object 
 1   price                      27508 non-null  float64
 2   id                         27508 non-null  float64
 3   developer                  27508 non-null  object 
 4   release_year               27508 non-null  object 
 5   Accounting                 27508 non-null  int64  
 6   Action                     27508 non-null  int64  
 7   Adventure                  27508 non-null  int64  
 8   Animation &amp; Modeling   27508 non-null  int64  
 9   Audio Production           27508 non-null  int64  
 10  Casual                     27508 non-null  int64  
 11  Design &amp; Illustration  27508 non-null  int64  
 12  Early Access               27508 non-null  int64  
 13  Education                  27508 non-null  int

In [38]:
df_steam_games.columns

Index(['app_name', 'price', 'id', 'developer', 'release_year', 'Accounting',
       'Action', 'Adventure', 'Animation &amp; Modeling', 'Audio Production',
       'Casual', 'Design &amp; Illustration', 'Early Access', 'Education',
       'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing',
       'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports',
       'Strategy', 'Utilities', 'Video Production', 'Web Publishing'],
      dtype='object')

#### Unificación de nombres de columnas y tipos de datos

Arreglo los nombre de las columnas. Unifico mayúsculas iniciales

In [45]:
df_steam_games.columns = ['Name', 'Price', 'Game_id', 'Developer', 'Release_year', 'Accounting',
       'Action', 'Adventure', 'Animation & Modeling', 'Audio Production',
       'Casual', 'Design & Illustration', 'Early Access', 'Education',
       'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing',
       'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports',
       'Strategy', 'Utilities', 'Video Production', 'Web Publishing']

In [48]:
# Se cambia el tipo de dato de Release_year
df_steam_games["Release_year"] = df_steam_games["Release_year"].astype(int)

#### Se exporta el dataframe modificado a un archivo parquet:

In [49]:
df_steam_games.to_parquet('steam_games.parquet', engine="pyarrow")

### 2. Archivo user_reviews.json
El procedimiento que utiilzo inicialmente es abrir el archivo usando el módulo ast de python, almacenar la info en un DataFrame de pandas, para luego inspeccionar los datos, eliminar registros con datos nulos y repetidos. Analizar otros cambios o transformaciones que sea necesario realizar, definir tipo de datos de las variables, eliminar columnas que no necesite.

In [41]:
"""Material de apoyo.mdPara leer el archivo user_reviews, que estaba presentado error al leerlo con pandas,
usamos el módulo ast de python """
data_list = []

# Ruta del archivo JSON
file_path = 'Dataset/user_reviews.json'

# Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

# Crear un DataFrame a partir de la lista de diccionarios
df_reviews = pd.DataFrame(data_list)

#### Observación del contenido extraído del archivo

In [42]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [43]:
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


Como veo que el archivo tiene el campo 'reviews' que es un json anidado, utilizo otro procedimiento diferente para leer el archivo inicial y desanidar los datos, empleando pd.explode y pd.json_normalize

In [44]:
# A continuación se crea una función que ayudará con la carga de la información en DataFrames de Pandas
def cargar_df(ruta, variable_anidada):
    '''Función que recibe una ruta de acceso a un archivo json anidado y carga la información en un
    DataFrame de Pandas'''
    rows = []
    with open(ruta, 'r', encoding='utf-8') as file:                         # se lee el archivo iterando línea por línea y agregando a lista vacia
        for line in file:
            rows.append(ast.literal_eval(line)) 
    
    df = pd.DataFrame(rows)                                                 # se carga la info en un df de Pandas            
    df = df.explode(variable_anidada).reset_index()                         # se separan en filas los datos anidados y se resetea index
    df = df.drop(columns="index")                                           # se elimina el indice original
    df = pd.concat([df, pd.json_normalize(df[variable_anidada])], axis=1)   # se realiza la apertura en columnas de la informacion anidada
    df = df.drop(columns=variable_anidada)                                  # se elimina la columna anidada original

    return df

In [45]:
# Se aplica la función anterior para abrir el archivo y cargar el df correspondiente a user reviews:
df_reviews = cargar_df(r'Dataset/user_reviews.json', "reviews")
df_reviews.shape

(59333, 9)

#### De nuevo observo el contenido extraído del archivo

In [46]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   user_url     59333 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  object
 8   review       59305 non-null  object
dtypes: object(9)
memory usage: 4.1+ MB


In [47]:
df_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [48]:
df_reviews.tail()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D
59332,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 2.,,440,No ratings yet,True,so much fun :D


#### Inspección de valores nulos

In [49]:
# Hago un conteo de valores nulos por columna
df_reviews.isnull().sum()

user_id         0
user_url        0
funny          28
posted         28
last_edited    28
item_id        28
helpful        28
recommend      28
review         28
dtype: int64

In [50]:
""" Veo que los valores nulos son pocos en cada columna, entonces decido eliminarlos todos pues la cantidad
de información que voy a perder no es signifcativa """
df_reviews = df_reviews.dropna().reset_index(drop=True)
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  object
 8   review       59305 non-null  object
dtypes: object(9)
memory usage: 4.1+ MB


In [51]:
df_reviews2=df_reviews.copy()

#### Revisión de duplicados

In [52]:
#hago un conteo de registros duplicados
has_duplicates = df_reviews2.duplicated().sum()
has_duplicates

874

In [53]:
#se observa el contenido de los duplicados
df_reviews2[df_reviews2.duplicated()]

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
1112,bokkkbokkk,http://steamcommunity.com/id/bokkkbokkk,,"Posted September 24, 2015.",,346110,1 of 1 people (100%) found this review helpful,True,yep
2891,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,,"Posted January 10, 2014.",,218620,1 of 3 people (33%) found this review helpful,True,"Good graphics, fun heists! A bit laggy"
2892,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,,"Posted January 10, 2014.",,105600,0 of 2 people (0%) found this review helpful,True,So fun! DEFINITELY NOT RIP OFF OF MINECRAFT! e...
2893,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,,"Posted December 17, 2014.",,570,No ratings yet,True,bobo pinoy
2894,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,,"Posted January 13, 2014.",,211820,No ratings yet,True,If you want to play this game.. expect glithes...
...,...,...,...,...,...,...,...,...,...
44433,76561198092022514,http://steamcommunity.com/profiles/76561198092...,,Posted July 3.,,422400,No ratings yet,True,Muy entretenido y una coleccion de armas prome...
44434,76561198092022514,http://steamcommunity.com/profiles/76561198092...,,Posted June 1.,,218620,No ratings yet,True,"Tiene una jugabilidad y tematica muy buena :D,..."
44435,76561198092022514,http://steamcommunity.com/profiles/76561198092...,,"Posted August 17, 2014.",,261820,No ratings yet,True,"Buen juego, no importa el desarrrollo que tien..."
44436,76561198092022514,http://steamcommunity.com/profiles/76561198092...,,"Posted February 17, 2014.",,224260,No ratings yet,True,exelente aporte :D¡¡¡ es una buen mod basado e...


In [54]:
# Se controlan varios de los casos para comprobar que se trate de valores duplicados
mascara = (df_reviews2["user_id"] == "ImSeriouss")
df_reviews2[mascara]

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
2885,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,,"Posted January 10, 2014.",,218620,1 of 3 people (33%) found this review helpful,True,"Good graphics, fun heists! A bit laggy"
2886,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,,"Posted January 10, 2014.",,105600,0 of 2 people (0%) found this review helpful,True,So fun! DEFINITELY NOT RIP OFF OF MINECRAFT! e...
2887,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,,"Posted December 17, 2014.",,570,No ratings yet,True,bobo pinoy
2888,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,,"Posted January 13, 2014.",,211820,No ratings yet,True,If you want to play this game.. expect glithes...
2889,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,,"Posted January 10, 2014.",,440,No ratings yet,True,Really good game! fun! Good for people who wan...
2890,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,,"Posted March 19, 2012.",,42680,No ratings yet,True,Good but a bit overdone. Still love it though.
2891,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,,"Posted January 10, 2014.",,218620,1 of 3 people (33%) found this review helpful,True,"Good graphics, fun heists! A bit laggy"
2892,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,,"Posted January 10, 2014.",,105600,0 of 2 people (0%) found this review helpful,True,So fun! DEFINITELY NOT RIP OFF OF MINECRAFT! e...
2893,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,,"Posted December 17, 2014.",,570,No ratings yet,True,bobo pinoy
2894,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,,"Posted January 13, 2014.",,211820,No ratings yet,True,If you want to play this game.. expect glithes...


In [55]:
mascara = (df_reviews2["user_id"] == "bokkkbokkk")
df_reviews2[mascara]

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
1111,bokkkbokkk,http://steamcommunity.com/id/bokkkbokkk,,"Posted September 24, 2015.",,346110,1 of 1 people (100%) found this review helpful,True,yep
1112,bokkkbokkk,http://steamcommunity.com/id/bokkkbokkk,,"Posted September 24, 2015.",,346110,1 of 1 people (100%) found this review helpful,True,yep


In [56]:
mascara = (df_reviews2["user_id"] == "76561198092022514")
df_reviews2[mascara]

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
21567,76561198092022514,http://steamcommunity.com/profiles/76561198092...,,Posted July 3.,,422400,No ratings yet,True,Muy entretenido y una coleccion de armas prome...
21568,76561198092022514,http://steamcommunity.com/profiles/76561198092...,,Posted June 1.,,218620,No ratings yet,True,"Tiene una jugabilidad y tematica muy buena :D,..."
21569,76561198092022514,http://steamcommunity.com/profiles/76561198092...,,"Posted August 17, 2014.",,261820,No ratings yet,True,"Buen juego, no importa el desarrrollo que tien..."
21570,76561198092022514,http://steamcommunity.com/profiles/76561198092...,,"Posted February 17, 2014.",,224260,No ratings yet,True,exelente aporte :D¡¡¡ es una buen mod basado e...
44433,76561198092022514,http://steamcommunity.com/profiles/76561198092...,,Posted July 3.,,422400,No ratings yet,True,Muy entretenido y una coleccion de armas prome...
44434,76561198092022514,http://steamcommunity.com/profiles/76561198092...,,Posted June 1.,,218620,No ratings yet,True,"Tiene una jugabilidad y tematica muy buena :D,..."
44435,76561198092022514,http://steamcommunity.com/profiles/76561198092...,,"Posted August 17, 2014.",,261820,No ratings yet,True,"Buen juego, no importa el desarrrollo que tien..."
44436,76561198092022514,http://steamcommunity.com/profiles/76561198092...,,"Posted February 17, 2014.",,224260,No ratings yet,True,exelente aporte :D¡¡¡ es una buen mod basado e...


Puedo concluír que efectivamente se trata de valores duplicados, por lo que decido eliminarlos todos

In [57]:
df_reviews2 = df_reviews2.drop_duplicates()
df_reviews2.shape

(58431, 9)

#### Elimino columnas que no necesito para responder los endpoints: user_url, funny, last_edited, helpful

In [58]:
df_reviews2 = df_reviews2.drop(columns=["user_url", "funny", "last_edited", "helpful"])
df_reviews2.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...


#### Revisión columna posted

De la fecha completa solo voy a necesitar el año, entonces extraigo ese dato en una nueva columna y elimino la columna con las fechas completas

In [59]:
df_reviews2["posted_year"] = df_reviews2["posted"].str.extract(r'(\d{4})')

In [60]:
df_reviews2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58431 entries, 0 to 59304
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      58431 non-null  object
 1   posted       58431 non-null  object
 2   item_id      58431 non-null  object
 3   recommend    58431 non-null  object
 4   review       58431 non-null  object
 5   posted_year  48498 non-null  object
dtypes: object(6)
memory usage: 3.1+ MB


#### Unificación de nombres de columnas y tipos de datos

Unifico nombres de columnas con las del DataFrame de los Games, además pongo mayúsuculas iniciales

In [61]:
df_reviews2.columns

Index(['user_id', 'posted', 'item_id', 'recommend', 'review', 'posted_year'], dtype='object')

In [62]:
df_reviews2.columns= ['User_id', 'Posted', 'Game_id', 'Recommend', 'Review', 'Posted_year']

Detecto que hay valores faltantes en la columna Posted year, entonces reviso cuáles son

In [63]:
df_reviews2[df_reviews2["Posted_year"].isna()]

,User_id,Posted,Game_id,Recommend,Review,Posted_year
6,evcentric,Posted February 3.,248820,True,A suitably punishing roguelike platformer. Wi...,NaN
27,76561198079601835,Posted May 20.,730,True,ZIKA DO BAILE,NaN
28,MeaTCompany,Posted July 24.,730,True,BEST GAME IN THE BLOODY WORLD,NaN
31,76561198156664158,Posted June 16.,252950,True,love it,NaN
32,76561198077246154,Posted June 11.,440,True,mt bom,NaN
...,...,...,...,...,...,...
59300,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...,NaN
59301,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...,NaN
59302,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,NaN
59303,LydiaMorley,Posted July 20.,730,True,:D,NaN


Aunque no tengo cómo conocer el año en que el review fue publicado, decido asignarle el año de lanzamiento del juego, pues es un numero muy grande de registros y no quiero eliminarlos. Entonces hago un join con el Dataframe que contiene esa información

In [64]:
# Se cambia el tipo de dato del Game id
df_reviews2["Game_id"] = df_reviews2["Game_id"].astype(float)

In [65]:
df_reviews_merged = pd.merge(df_reviews2, df_steam_games, on = "Game_id", how= "inner")

In [66]:
# Reemplazo valores nulos en la columna Posted-year
df_reviews_merged["Posted_year"].fillna(df_reviews_merged["Release_year"], inplace = True)

In [67]:
df_reviews_merged.isnull().sum()

User_id                    0
Posted                     0
Game_id                    0
Recommend                  0
Review                     0
Posted_year                0
Name                       0
Price                      0
Developer                  0
Release_year               0
Accounting                 0
Action                     0
Adventure                  0
Animation_and_Modeling     0
Audio_Production           0
Casual                     0
Design_and_Illustration    0
Early_Access               0
Education                  0
Free_to_Play               0
Indie                      0
Massively_Multiplayer      0
Photo_Editing              0
RPG                        0
Racing                     0
Simulation                 0
Software_Training          0
Sports                     0
Strategy                   0
Utilities                  0
Video_Production           0
Web_Publishing             0
dtype: int64

In [68]:
df_reviews_merged.columns

Index(['User_id', 'Posted', 'Game_id', 'Recommend', 'Review', 'Posted_year',
       'Name', 'Price', 'Developer', 'Release_year', 'Accounting', 'Action',
       'Adventure', 'Animation_and_Modeling', 'Audio_Production', 'Casual',
       'Design_and_Illustration', 'Early_Access', 'Education', 'Free_to_Play',
       'Indie', 'Massively_Multiplayer', 'Photo_Editing', 'RPG', 'Racing',
       'Simulation', 'Software_Training', 'Sports', 'Strategy', 'Utilities',
       'Video_Production', 'Web_Publishing'],
      dtype='object')

In [69]:
# Descarto las columnas que no necesito
df_reviews_merged=df_reviews_merged.drop(columns=['Posted',
       'Name', 'Price', 'Developer', 'Release_year', 'Accounting', 'Action',
       'Adventure', 'Animation_and_Modeling', 'Audio_Production', 'Casual',
       'Design_and_Illustration', 'Early_Access', 'Education', 'Free_to_Play',
       'Indie', 'Massively_Multiplayer', 'Photo_Editing', 'RPG', 'Racing',
       'Simulation', 'Software_Training', 'Sports', 'Strategy', 'Utilities',
       'Video_Production', 'Web_Publishing'])

In [70]:
df_reviews3=df_reviews_merged.copy()
df_reviews3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48949 entries, 0 to 48948
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   User_id      48949 non-null  object 
 1   Game_id      48949 non-null  float64
 2   Recommend    48949 non-null  object 
 3   Review       48949 non-null  object 
 4   Posted_year  48949 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.9+ MB


In [71]:
df_reviews3.head()

,User_id,Game_id,Recommend,Review,Posted_year
0,76561197970982479,1250.0,True,Simple yet with great replayability. In my opi...,2011
1,death-hunter,1250.0,True,"Amazing, Non-stop action of blowing stuff to b...",2015
2,DJKamBer,1250.0,True,"Compared to Left 4 Dead 2, this game REALLY gi...",2013
3,diego9031,1250.0,True,Jogo ♥♥♥♥.,2015
4,76561198081962345,1250.0,True,cara nas imagens esse jogo da pouco de medo ma...,2014


In [72]:
# Modifico el tipo de dato del año de publicación para que sea integer
df_reviews3['Posted_year']=df_reviews3['Posted_year'].astype(int)

La columna Recommend es de tipo boolenao y prefiero convertirla en tipo integer

In [73]:
df_reviews3['Recommend']=df_reviews3['Recommend'].astype(int)

In [74]:
df_reviews3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48949 entries, 0 to 48948
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   User_id      48949 non-null  object 
 1   Game_id      48949 non-null  float64
 2   Recommend    48949 non-null  int64  
 3   Review       48949 non-null  object 
 4   Posted_year  48949 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.9+ MB


#### Creación de columna Sentiment Analysis con NLP
Entre los requerimientos del proyecto se encuentra crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: 
- debe tomar el valor '0' si es malo 
- '1' si es neutral o no existe reseña escrita
- '2' si es positivo
Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos

In [75]:
# En primer lugar, se importan las herramientas que nos ayudarán en la tarea
import nltk
import ssl

from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [76]:
""" Luego se crea una función que devolverá las clasificaciones mencionadas anteriormente, 
en función al score obtenido por el algoritmo SentimentIntensityAnalyzer() de la biblioteca nltk """
def classify_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()         # se instancia el algoritmo
    if pd.isnull(text) or text == '':               # si la celda es null o el texto es una cadena vacía
        return 1
    elif isinstance(text, str):                         # sino, si se trata de una cadena de texto válida
        sentiment = analyzer.polarity_scores(text)      # se obtienen los scores de sentimiento
        compound_score = sentiment['compound']          # se selecciona el score "compound"

        # Se califica el texto en función al score
        if compound_score >= 0.05:
            return 2
        elif compound_score <= (-0.05):
            return 0
        else:
            return 1


In [77]:
# Aplico la función creada anteriormente, a la columna reviews
df_reviews3['Sentiment_analysis'] = df_reviews3["Review"].apply(classify_sentiment)
df_reviews3.head()

,User_id,Game_id,Recommend,Review,Posted_year,Sentiment_analysis
0,76561197970982479,1250.0,1,Simple yet with great replayability. In my opi...,2011,2
1,death-hunter,1250.0,1,"Amazing, Non-stop action of blowing stuff to b...",2015,2
2,DJKamBer,1250.0,1,"Compared to Left 4 Dead 2, this game REALLY gi...",2013,0
3,diego9031,1250.0,1,Jogo ♥♥♥♥.,2015,1
4,76561198081962345,1250.0,1,cara nas imagens esse jogo da pouco de medo ma...,2014,1


In [78]:
# Hago una revisión de los valores para cada uno de las clasificaciones
df_reviews3['Sentiment_analysis'].value_counts()

Sentiment_analysis
2    31150
1     9976
0     7823
Name: count, dtype: int64

In [79]:
# Elimino la columna Reviews

df_reviews3 = df_reviews3.drop(columns=['Review'])

In [80]:
# Hago una última revisión de los tipos de datos
df_reviews3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48949 entries, 0 to 48948
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   User_id             48949 non-null  object 
 1   Game_id             48949 non-null  float64
 2   Recommend           48949 non-null  int64  
 3   Posted_year         48949 non-null  int64  
 4   Sentiment_analysis  48949 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 1.9+ MB


#### Se exporta el dataframe modificado, a un archivo parquet:

In [101]:
df_reviews3.to_parquet('user_reviews.parquet', engine="pyarrow")

### 3. Archivo user_items.json
Tras una exploración inicial y un intento de cargar la información de este archivo, veo que la columna items está anidada. Aplico la función utilizada con el archivo user reviews, para extraer y desanidar el contenido de este archivo

In [81]:
df_items = cargar_df(r'Dataset/users_items.json', "items")
df_items.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0


In [122]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 315.6+ MB


#### Inspección de valores nulos

In [82]:
# Hago un conteo de valores nulos por columna
df_items.isnull().sum()

user_id                 0
items_count             0
steam_id                0
user_url                0
item_id             16806
item_name           16806
playtime_forever    16806
playtime_2weeks     16806
dtype: int64

In [83]:
""" Dado que la proporción de valores nula es muy baja con respecto al total de registros, decido eliminarlos
sin hacer ninguna revisión adicional """
df_items = df_items.dropna().reset_index(drop=True)
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 314.5+ MB


#### Inspección de valores duplicados

In [86]:
# Hago un conteo de registros duplicados
has_duplicates = df_items.duplicated().sum()
has_duplicates

59104

In [1]:
""" Dado que el porcentaje de filas duplicadas es muy pequeño con respecto al número total de registros,
decido eliminarlas, pues la cantidad de información que voy a perder no es signifcativa """
df_items = df_items.drop_duplicates()
df_items.info()

NameError: name 'df_items' is not defined

In [97]:
df_items2=df_items.copy()

#### Elimino columnas que no voy a utilizar : steam_id, user_url, playtime 2weeks. Reconstruyo el index

In [98]:
df_items2=df_items2.drop(columns=["steam_id", "user_url", "playtime_2weeks"])
df_items2.reset_index(inplace=True, drop=True)
df_items2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094105 entries, 0 to 5094104
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   item_id           object 
 3   item_name         object 
 4   playtime_forever  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 194.3+ MB


In [99]:
df_items2.tail(20)

,user_id,items_count,item_id,item_name,playtime_forever
5094085,76561198326700687,177,502550,Strike.is: The Game,0.0
5094086,76561198326700687,177,503010,Legends of Azulgar,0.0
5094087,76561198326700687,177,506140,Shop Heroes,0.0
5094088,76561198326700687,177,402570,Atlas Reactor,0.0
5094089,76561198326700687,177,508380,Amnesia: Final Revelations,0.0
5094090,76561198326700687,177,510050,You Have 10 Seconds,0.0
5094091,76561198326700687,177,510660,Big Bang Empire,0.0
5094092,76561198326700687,177,512470,The Pirate: Caribbean Hunt,0.0
5094093,76561198326700687,177,104900,ORION: Prelude,0.0
5094094,76561198326700687,177,516510,Orake 2D MMORPG,0.0


#### Unifico nombres de las columnas y tipo de datos con los archivos anteriores

In [116]:
df_items2.columns =["User_id", "Items_count", "Game_id", "Name", "Playtime"]

In [118]:
df_items2["Game_id"]=df_items2["Game_id"].astype(float)

In [119]:
df_items2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094105 entries, 0 to 5094104
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   User_id      object 
 1   Items_count  int64  
 2   Game_id      float64
 3   Name         object 
 4   Playtime     float64
dtypes: float64(2), int64(1), object(2)
memory usage: 194.3+ MB


#### Exporto el dataframe normalizado a un archivo parquet

In [120]:
df_items2.to_parquet('user_items.parquet', engine="pyarrow")